In [971]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

Получение данных

In [79]:
data = pd.read_excel(io='case_dataset.xlsx', engine='openpyxl', sheet_name=['case_dataset'])
data = data['case_dataset']

In [80]:
spheres = data['sphere'].unique()
df_res = pd.DataFrame()
for sphere in tqdm(spheres):
    this_sphere = data[data['sphere'] == sphere]
    df_res = df_res.append(this_sphere, ignore_index=True)

  0%|          | 0/34 [00:00<?, ?it/s]

Предобработка

In [81]:
from string import punctuation
import re

In [82]:
def remove_punctuation(text):
    return "".join([ch if ch not in punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

In [83]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(str(text).lower()))) for text in tqdm(df_res['text'])]
df_res['prep_text'] = prep_text

  0%|          | 0/8066 [00:00<?, ?it/s]

Стемминг

In [86]:
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords

In [931]:
stemmer = SnowballStemmer("russian")
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'здравствуйте'])

In [88]:
stemmed_texts = []
for text in tqdm(df_res['prep_text']):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts.append(text)

df_res['stem_text'] = stemmed_texts

  0%|          | 0/8066 [00:00<?, ?it/s]

Лемматизация

In [ ]:
import pymorphy2
import nltk
morph = pymorphy2.MorphAnalyzer()

lemm_texts_list = []
for text in tqdm(df_res['prep_text']):
    try:
        sentence_words = nltk.word_tokenize(text)
        text_lem = []
        for i in sentence_words:
            text_lem.append(morph.parse(i)[0].normal_form)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e.__name__)
df_res['lemm_text'] = lemm_texts_list

Выбор 12 сфер


In [507]:
counter = {}
for i in data['sphere']:
    if i != 'Прочее':
        counter[i] = counter.get(i, 0) + 1
count = sorted([[j, i] for i, j in counter.items()], reverse=True)
spheres = [i[1] for i in count[:12]]

In [514]:
df = pd.DataFrame()
for sphere in tqdm(spheres):
    this_sphere = df_res[df_res['sphere'] == sphere]
    df = df.append(this_sphere, ignore_index=True)

  0%|          | 0/12 [00:00<?, ?it/s]

In [515]:
df.to_csv('format_text.csv', encoding='utf-8')

Перевод сфер из строк в цифры


In [516]:
dict_spheres = {}
for i in range(len(spheres)):
    dict_spheres[spheres[i]] = i
df['sphere'] = [dict_spheres[i] for i in df['sphere']]

Классификация

In [517]:
# df = pd.read_csv('format_text.csv')
df = df.drop(columns=['category', 'theme', 'text', 'prep_text', 'stem_text'])

In [520]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import f1_score, classification_report

In [573]:
X = df['lemm_text']
y = df['sphere']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

print(f"Количество данных в y_train по классам: {np.bincount(y_train)}")
print(f"Количество данных в y_test по классам: {np.bincount(y_test)}")

Количество данных в y_train по классам: [1488 1197 1112 1018  271  173  168  159   90   88   81   75]
Количество данных в y_test по классам: [372 299 278 255  68  43  42  40  23  22  20  19]


LogisticRegression

In [950]:
from sklearn.linear_model import LogisticRegression
lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(solver='saga'))])

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8102467111517623
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.72      0.86      0.79       372
                                                                                  Здравоохранение       0.93      0.97      0.95       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.71      0.79      0.75       278
                                                                             Автомобильные дороги       0.85      0.85      0.85       255
                                                                 Социальное обеспечение населения       0.67      0.63      0.65        68
                                                                      Общественный транспорт (ОТ)       0.88      0.67      0.76        4

для презентации

In [ ]:
df_res['text'][6]

In [ ]:
list(df_res['lemm_text'])

In [ ]:
np.set_printoptions(threshold=1000)
lr['vect'].fit_transform(list(df['lemm_text'])[2:7])[4].toarray()

In [ ]:
np.set_printoptions(threshold=1000)
T = lr['vect'].fit_transform(list(df['lemm_text'])[4:7]).toarray()
lr['tfidf'].fit_transform(T)[2].toarray()

PassiveAggressiveClassifier


In [694]:
from sklearn.linear_model import PassiveAggressiveClassifier
PAC = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', PassiveAggressiveClassifier(max_iter=5000, early_stopping=True, validation_fraction=0.2, n_iter_no_change=3))])

PAC.fit(X_train, y_train)
y_pred = PAC.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8192588474798953
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.85      0.82      0.83       372
                                                                                  Здравоохранение       0.94      0.96      0.95       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.73      0.76      0.74       278
                                                                             Автомобильные дороги       0.78      0.84      0.81       255
                                                                 Социальное обеспечение населения       0.72      0.71      0.71        68
                                                                      Общественный транспорт (ОТ)       0.88      0.88      0.88        4

Perceptron


In [696]:
from sklearn.linear_model import Perceptron
prc = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', Perceptron(max_iter=5000, early_stopping=True, validation_fraction=0.1, n_iter_no_change=8))])

prc.fit(X_train, y_train)
y_pred = prc.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8028951044078295
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.85      0.83      0.84       372
                                                                                  Здравоохранение       0.94      0.95      0.94       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.75      0.70      0.72       278
                                                                             Автомобильные дороги       0.77      0.85      0.80       255
                                                                 Социальное обеспечение населения       0.63      0.65      0.64        68
                                                                      Общественный транспорт (ОТ)       0.73      0.88      0.80        4

RidgeClassifier

In [711]:
from sklearn.linear_model import RidgeClassifier
ridge = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RidgeClassifier())])

ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8278100440011483
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.80      0.86      0.83       372
                                                                                  Здравоохранение       0.93      0.97      0.95       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.73      0.77      0.75       278
                                                                             Автомобильные дороги       0.82      0.84      0.83       255
                                                                 Социальное обеспечение населения       0.71      0.71      0.71        68
                                                                      Общественный транспорт (ОТ)       0.90      0.86      0.88        4

SGDClassifier


In [749]:
from sklearn.linear_model import SGDClassifier
sgd = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', SGDClassifier(max_iter=5000))])

sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8257827398530792
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.82      0.85      0.83       372
                                                                                  Здравоохранение       0.94      0.96      0.95       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.74      0.74      0.74       278
                                                                             Автомобильные дороги       0.81      0.85      0.83       255
                                                                 Социальное обеспечение населения       0.68      0.74      0.71        68
                                                                      Общественный транспорт (ОТ)       0.86      0.88      0.87        4

ComplementNB


In [786]:
from sklearn.naive_bayes import ComplementNB
complement = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', ComplementNB())])

complement.fit(X_train, y_train)
y_pred = complement.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8068382960409166
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.81      0.84      0.83       372
                                                                                  Здравоохранение       0.87      0.99      0.93       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.76      0.71      0.73       278
                                                                             Автомобильные дороги       0.73      0.89      0.80       255
                                                                 Социальное обеспечение населения       0.62      0.68      0.65        68
                                                                      Общественный транспорт (ОТ)       0.91      0.67      0.77        4

KNeighborsClassifier


In [840]:
from sklearn.neighbors import KNeighborsClassifier
knc = Pipeline([('vect', CountVectorizer()),
                       ('tfidf', TfidfTransformer()),
                       ('clf', KNeighborsClassifier(n_neighbors=13))])

knc.fit(X_train, y_train)
y_pred = knc.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8062593678270484
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.79      0.86      0.83       372
                                                                                  Здравоохранение       0.93      0.98      0.96       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.75      0.67      0.71       278
                                                                             Автомобильные дороги       0.70      0.87      0.78       255
                                                                 Социальное обеспечение населения       0.66      0.68      0.67        68
                                                                      Общественный транспорт (ОТ)       0.79      0.72      0.76        4

MLPClassifier


In [864]:
from sklearn.neural_network import MLPClassifier
mlp = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MLPClassifier(solver='adam', max_iter=200, early_stopping=True, n_iter_no_change=8, validation_fraction=0.15))])

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8136395795384818
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.81      0.84      0.82       372
                                                                                  Здравоохранение       0.95      0.97      0.96       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.69      0.77      0.73       278
                                                                             Автомобильные дороги       0.80      0.83      0.81       255
                                                                 Социальное обеспечение населения       0.61      0.78      0.68        68
                                                                      Общественный транспорт (ОТ)       0.92      0.77      0.84        4

RandomForestClassifier


In [901]:
from sklearn.ensemble import RandomForestClassifier
rf = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(bootstrap=False))])

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8081703421393587
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.76      0.86      0.81       372
                                                                                  Здравоохранение       0.91      0.97      0.94       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.74      0.75      0.75       278
                                                                             Автомобильные дороги       0.77      0.88      0.82       255
                                                                 Социальное обеспечение населения       0.63      0.57      0.60        68
                                                                      Общественный транспорт (ОТ)       0.85      0.81      0.83        4

StackingClassifier


In [923]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression

estimators = [('rf', RandomForestClassifier(bootstrap=False)),
              ('ridge', RidgeClassifier())]
stack = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(solver='saga')))])

stack.fit(X_train, y_train)
y_pred = stack.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8340564158153034
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.83      0.85      0.84       372
                                                                                  Здравоохранение       0.95      0.96      0.96       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.73      0.77      0.75       278
                                                                             Автомобильные дороги       0.83      0.85      0.84       255
                                                                 Социальное обеспечение населения       0.74      0.78      0.76        68
                                                                      Общественный транспорт (ОТ)       0.88      0.86      0.87        4

VotingClassifier


In [952]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

estimators = [('ridge', RidgeClassifier()),
              ('lr', LogisticRegression(solver='saga')),
              ('sgd', SGDClassifier(max_iter=5000)),
              ('rf', RandomForestClassifier(bootstrap=False))]
voting = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', VotingClassifier(estimators=estimators))])

voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)

print('f1_score %s' % f1_score(y_pred, y_test, average='weighted'))
print(classification_report(y_test, y_pred, target_names=spheres))

f1_score 0.8205263316147104
                                                                                                   precision    recall  f1-score   support

                                                                   Жилищно-коммунальное хозяйство       0.75      0.87      0.80       372
                                                                                  Здравоохранение       0.93      0.97      0.95       299
Благоустройство населённых пунктов (площади, парки, скверы, улицы, дворовые территории, кладбища)       0.74      0.78      0.76       278
                                                                             Автомобильные дороги       0.85      0.85      0.85       255
                                                                 Социальное обеспечение населения       0.67      0.68      0.67        68
                                                                      Общественный транспорт (ОТ)       0.90      0.81      0.85        4